In [1]:
import agate

In [2]:
specified_types = {
    'CAMPUS': agate.Text()
}

students0607 = agate.Table.from_csv('../raw_data/0607student.txt')
students1516 = agate.Table.from_csv('../raw_data/1516student.txt')
cr0607 = agate.Table.from_csv('../raw_data/0607cr.txt')
cr1516 = agate.Table.from_csv('../raw_data/1516cr.txt')
locations = agate.Table.from_csv('../raw_data/Current_Schools.csv')

In [3]:
join0607 = students0607.join(cr0607, left_key = 'CAMPUS', full_outer = 'true', require_match = 'false', columns = 'true')
join1516 = students1516.join(cr1516, left_key = 'CAMPUS', full_outer = 'true', require_match = 'false', columns = 'true')


In [4]:
columns_one = ['CAMPUS', 'CPETALLC','CPETBLAC', 'CPETHISC', 'CPETWHIC', 'CPETINDC', 'CPETPACC',
        'CPETECOC', 'CPETLEPC','CPETRSKC', 'CPETBLAP','CPETHISP','CPETWHIP', 'CPETINDP', 'CPETPACP', 'CPETECOP','CPETLEPP',
'CPETRSKP','CPETBILC', 'DISTNAME', 'CAMPNAME', 'CNTYNAME']

columns_three = ['CAMPUS', 'CPETALLC','CPETBLAC', 'CPETHISC', 'CPETWHIC', 'CPETINDC', 'CPETASIC', 'CPETPCIC', 'CPETTWOC',
       'CPETECOC', 'CPETLEPC','CPETBLAP','CPETHISP','CPETWHIP','CPETINDP', 'CPETASIP', 'CPETPCIP', 'CPETTWOP', 'CPETECOP','CPETLEPP', 'DISTNAME', 'CAMPNAME', 'CNTYNAME']
 
campus0607_select = join0607.select(columns_one)
campus1516_select = join1516.select(columns_three)

In [24]:
column_names_one = {'CAMPUS': 'CAMPUS',
                                       'CPETALLC': 'ALL_0607',
                                       'CPETBLAC': 'B_0607',
                                       'CPETHISC': 'H_0607',
                                       'CPETWHIC': 'W_0607',
                                       'CPETINDC': 'AI_0607',
                                       'CPETPACC': 'API_0607',
                                       'CPETECOC': 'EC_0607',
                                       'CPETLEPC': 'EL_0607',
                                       'CPETRSKC': 'AT_0607',
                                       'CPETBLAP': 'BP_0607',
                                       'CPETHISP': 'HP_0607',
                                       'CPETWHIP': 'WP_0607',
                                       'CPETINDP': 'AIP_0607',
                                       'CPETPACP': 'APIP_0607',
                                       'CPETECOP': 'ECP_0607',
                                       'CPETLEPP': 'ELP_0607',
                                       'CPETRSKP': 'ATP_0607',
                                       'CPETBILC': 'BILINGUAL_ENROLL'}
column_names_three = {'CAMPUS': 'CAMPUS',
                                       'CPETALLC': 'ALL_1516',
                                       'CPETBLAC': 'B_1516',
                                       'CPETHISC': 'H_1516',
                                       'CPETWHIC': 'W_1516',
                                       'CPETINDC': 'AI_1516',
                                       'CPETASIC': 'AS_1516',
                                       'CPETPCIC': 'PI_1516',
                                       'CPETTWOC': 'TWO_1516',
                                       'CPETECOC': 'EC_1516',
                                       'CPETLEPC': 'EL_1516',
                                       'CPETBLAP': 'BP_1516',
                                       'CPETHISP': 'HP_1516',
                                       'CPETWHIP': 'WP_1516',
                                       'CPETINDP': 'AIP_1516',
                                       'CPETASIP': 'ASP_1516',
                                       'CPETPCIP': 'PIP_1516',
                                       'CPETTWOP': 'TWOP_1516',                
                                       'CPETECOP': 'ECP_1516',
                                       'CPETLEPP': 'ELP_1516',
                                       'CPETBILC': 'BILINGUAL_ENROLL'}
campus0607_rename = campus0607_select.rename(column_names_one)
campus1516_rename = campus1516_select.rename(column_names_three)

In [25]:
class datecolumn(agate.Computation): 
    def get_computed_data_type(self, table):
        return agate.Text()
    def run(self, table):
        new_column = []
        for row in table.rows:
            new_column.append(year_value)
        return new_column
    
year_value = '2007-01-01'
campus0607_date = campus0607_rename.compute([
    ('year', datecolumn())
])

In [26]:
year_value = '2016-01-01'
campus1516_date = campus1516_rename.compute([
    ('year', datecolumn())
])

In [27]:
campus_full = agate.Table.merge([campus0607_date, campus1516_date])

## Now filter to needed districts.

district_ids = ['AUSTIN ISD', 'ROUND ROCK ISD', 'LEANDER ISD', 'PFLUGERVILLE ISD', 'HAYS CISD', 'DEL VALLE ISD', 'GEORGETOWN ISD', 'BASTROP ISD', 'MANOR ISD', 'LAKE TRAVIS ISD', 'EANES ISD', 'SAN MARCOS CISD', 'HUTTO ISD', 'DRIPPING SPRINGS ISD', 'ELGIN ISD']

campus_filtertwo = campus_full.where(lambda row: row['DISTNAME'].upper() in district_ids
                              )

campus_filtertwo.to_csv(path='campus_filtertwo.csv')

In [28]:
located = campus_filtertwo.join(locations, 'CAMPUS')

In [29]:
latlong = {'\ufeffX': 'longitude',
          'Y': 'latitude'}
located = located.rename(latlong)


In [32]:
location_columns = ['DISTNAME', 'CAMPNAME', 'CNTYNAME', 'longitude', 'latitude','STREET', 'CITY', 'ZIP','CAMPUS',
                    'ALL_0607','ALL_1516','B_0607','B_1516','H_0607','H_1516','W_0607','W_1516','AI_0607','AI_1516',
                    'API_0607','AS_1516','PI_1516','TWO_1516','EC_0607','EC_1516','EL_0607','EL_1516','AT_0607',
                    'BP_0607','BP_1516','HP_0607','HP_1516','WP_0607','WP_1516','AIP_0607','AIP_1516',
                    'APIP_0607','ASP_1516','PIP_1516','TWOP_1516','ECP_0607','ECP_1516','ELP_0607','ELP_1516','ATP_0607']

In [33]:
locationfiltertwo = located.select(location_columns)
locationfiltertwo.to_csv(path='locationfiltertwo.csv')